# ELT Dieese

In [14]:
%run ../../config/bootstrap.py
from utils import get_project_root,build_row_hash
from pathlib import Path 
#from pandas_scd import scd2
import pandas as pd
from datetime import datetime

In [2]:

project_root = get_project_root() 
project_root

PosixPath('/Users/silmara.barnabe/Desktop/dengue')

## ⌛ Staging


In [3]:
df_stg = pd.read_excel(
    project_root / "data/staging/dieese/dieese_dataset_201001_202504.xls",
    engine="xlrd",
    header=1
)

In [4]:
df_stg.head()

,Unnamed: 0,Brasília,Campo Grande,Cuiabá,Goiânia,Belo Horizonte,Rio de Janeiro,São Paulo,Vitória,Curitiba,Florianópolis,Porto Alegre,Belém,Boa Vista,Macapá,Manaus,Palmas,Porto Velho,Rio Branco,Aracaju,Fortaleza,João Pessoa,Maceió,Natal,Recife,Salvador,São Luís,Teresina,Macaé
0,01-2010,214.47,NaN,NaN,199.71,205.69,213.77,225.02,217.20,211.99,213.23,236.55,204.62,NaN,NaN,216.53,NaN,NaN,NaN,169.13,175.86,171.97,NaN,186.72,172.29,185.77,NaN,NaN,NaN
1,02-2010,216.89,NaN,NaN,190.63,216.51,221.80,229.64,224.74,215.61,217.59,238.46,204.93,NaN,NaN,223.90,NaN,NaN,NaN,169.57,176.89,179.28,NaN,193.50,184.08,198.24,NaN,NaN,NaN
2,03-2010,236.41,NaN,NaN,206.91,224.37,240.22,253.74,232.23,231.30,229.80,257.07,215.72,NaN,NaN,231.31,NaN,NaN,NaN,181.70,182.43,196.29,NaN,199.14,202.01,208.71,NaN,NaN,NaN
3,04-2010,237.76,NaN,NaN,206.46,239.06,253.13,261.39,244.07,238.71,239.67,268.72,227.04,NaN,NaN,241.52,NaN,NaN,NaN,184.97,187.21,203.86,NaN,223.22,214.48,220.00,NaN,NaN,NaN
4,05-2010,233.25,NaN,NaN,212.07,240.47,240.36,256.31,242.85,233.49,235.89,256.86,221.74,NaN,NaN,249.39,NaN,NaN,NaN,187.10,185.73,200.09,NaN,219.57,216.85,216.18,NaN,NaN,NaN


## 🥉 Bronze

In [5]:

df_stg = df_stg.rename(columns={df_stg.columns[0]: "data"})  
df_bronze = (
    df_stg
      .melt(id_vars="data",          # mantém a coluna data
            var_name="municipio",    # novos rótulos das colunas “largas”
            value_name="vlr_cesta_basica")  # valores das antigas colunas
      .dropna(subset=["vlr_cesta_basica"])  # remove linhas sem valor
      .reset_index(drop=True)        # limpa o índice
)

# opcional: converte a coluna data para datetime se quiser filtrar por datas depois
df_bronze["data"] = "01-"+df_bronze["data"]

df_bronze.head()

,data,municipio,vlr_cesta_basica
0,01-01-2010,Brasília,214.47
1,01-02-2010,Brasília,216.89
2,01-03-2010,Brasília,236.41
3,01-04-2010,Brasília,237.76
4,01-05-2010,Brasília,233.25


In [ ]:
output_path = project_root / "data" / "bronze" / "dieese" / "dieese.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df_bronze.to_csv(output_path, index=False)


In [9]:
output_path

PosixPath('/Users/silmara.barnabe/Desktop/dengue/data/bronze/dieese/dieese.csv')

## 🥈 Silver

In [22]:
output_path = project_root / "data" / "bronze" / "dieese" / "dieese.csv"

In [23]:
df_bronze = pd.read_csv(output_path)
df_bronze.head()

,data,municipio,vlr_cesta_basica
0,01-01-2010,Brasília,214.47
1,01-02-2010,Brasília,216.89
2,01-03-2010,Brasília,236.41
3,01-04-2010,Brasília,237.76
4,01-05-2010,Brasília,233.25


In [24]:
df_bronze.municipio.unique()

array(['Brasília', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Belo Horizonte',
       'Rio de Janeiro', 'São Paulo', 'Vitória', 'Curitiba',
       'Florianópolis', 'Porto Alegre', 'Belém', 'Boa Vista', 'Macapá',
       'Manaus', 'Palmas', 'Porto Velho', 'Rio Branco', 'Aracaju',
       'Fortaleza', 'João Pessoa', 'Maceió', 'Natal', 'Recife',
       'Salvador', 'São Luís', 'Teresina', 'Macaé'], dtype=object)

In [25]:

city_to_ibge = {
    "Brasília":      5300108,
    "Campo Grande":  5002704,
    "Cuiabá":        5103403,
    "Goiânia":       5208707,
    "Belo Horizonte":3106200,
    "Rio de Janeiro":3304557,
    "São Paulo":     3550308,
    "Vitória":       3205309,
    "Curitiba":      4106902,
    "Florianópolis": 4205407,
    "Porto Alegre":  4314902,
    "Belém":         1501402,
    "Boa Vista":     1400100,
    "Macapá":        1600303,
    "Manaus":        1302603,
    "Palmas":        1721000,
    "Porto Velho":   1100205,
    "Rio Branco":    1200401,
    "Aracaju":       2800308,
    "Fortaleza":     2304400,
    "João Pessoa":   2507507,
    "Maceió":        2704302,
    "Natal":         2408102,
    "Recife":        2611606,
    "Salvador":      2927408,
    "São Luís":      2111300,
    "Teresina":      2211001,
    "Macaé":         3302403
}

In [26]:
df_silver = df_bronze.copy()
df_silver["id_municipio"] = df_bronze["municipio"].map(city_to_ibge)
df_silver = df_silver[['data','municipio','id_municipio','vlr_cesta_basica']]
df_silver.head()

,data,municipio,id_municipio,vlr_cesta_basica
0,01-01-2010,Brasília,5300108,214.47
1,01-02-2010,Brasília,5300108,216.89
2,01-03-2010,Brasília,5300108,236.41
3,01-04-2010,Brasília,5300108,237.76
4,01-05-2010,Brasília,5300108,233.25


### Quality

In [27]:
missing = df_silver.loc[df_silver["id_municipio"].isna(), "municipio"].unique()
if len(missing):
    print("Cidades sem código encontrado:", missing)


### hash

In [28]:
df_silver.columns

Index(['data', 'municipio', 'id_municipio', 'vlr_cesta_basica'], dtype='object')

In [29]:
cols_for_hash = ['data', 'municipio', 'id_municipio', 'vlr_cesta_basica']

In [30]:
df_silver["hash"] = build_row_hash(df_silver, cols_for_hash, algo="sha256")
df_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   data              3491 non-null   object 
 1   municipio         3491 non-null   object 
 2   id_municipio      3491 non-null   int64  
 3   vlr_cesta_basica  3491 non-null   float64
 4   hash              3491 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 136.5+ KB


In [31]:
df_silver.hash.nunique()

3491

### scd2

In [32]:
df_silver['start_date'] = datetime(2025, 5, 3)
df_silver['end_date'] =  None
df_silver['is_current'] =  True
df_silver.head()

,data,municipio,id_municipio,vlr_cesta_basica,hash,start_date,end_date,is_current
0,01-01-2010,Brasília,5300108,214.47,a97e54b050d2c821cd9b71998e7ca9a826b1c0f5f42c13d6bb76b9c3884fd17b,2025-05-03,None,True
1,01-02-2010,Brasília,5300108,216.89,47f62e0830a6ff778b3f692bd478f5dc06a555b845388c77f4c5a745d82c984a,2025-05-03,None,True
2,01-03-2010,Brasília,5300108,236.41,274088489c0d8a40792dde8f5b6868c1a8e74a7fc501556a6eebaea16a6820fb,2025-05-03,None,True
3,01-04-2010,Brasília,5300108,237.76,06a8ba8cf510665dc725de5ab84a312b880a67422140cc91a90990ce006afc91,2025-05-03,None,True
4,01-05-2010,Brasília,5300108,233.25,86acbad29b8c4315fee893cd99632f1d665575ada9a8ed52e36f114276fae494,2025-05-03,None,True


### save

In [34]:
output_path = project_root / "data" / "silver" / "dieese" / "dieese.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df_silver.to_csv(output_path, index=False)


## 🥇 Gold

In [35]:
silver_path = project_root / "data" / "silver" / "dieese" / "dieese.csv"
gold_path   = project_root / "data" / "gold"  / "dieese" / "dieese.csv"


gold_path.parent.mkdir(parents=True, exist_ok=True)

# df_silver = pd.read_csv(silver_path, parse_dates=["start_date", "end_date"])

# --- filtra registros vigentes ---
df_gold = df_silver.loc[df_silver["is_current"] == 1].copy()

# --- grava a gold ---
df_gold.to_csv(gold_path, index=False)

print(f"Salvo {len(df_gold):,} registros atuais em {gold_path}")


Salvo 3,491 registros atuais em /Users/silmara.barnabe/Desktop/dengue/data/gold/dieese/dieese.csv
